In [1]:
import sys
import os
%matplotlib inline

In [2]:
# get current working directory
os.getcwd()
# change directory
os.chdir('../')
os.getcwd()

'/home/romagnolilab/Documents/GitHub/ACTC_MD_ML'

In [3]:
# import libraries
from src.polymerSMILES import polymers
from src.dataProcessing import * #DataPreprocessing #computeTanimotoSimilarity #* #data_load, computeTanimotoSimilarity, get_dissimilarity_map, featurizer, normalizedata

In [4]:
# load smiles
polymers_dict = polymers.get_data() # get rdkit formats of the polymer smiles 


In [5]:
# instantiate data preprocessing toolbox
preprocess = DataPreprocessing()

In [6]:
# compute the tanimoto similarity
TanimotoIndex = preprocess.computeTanimotoSimilarity(polymers_dict, 'Tanimoto') 
#preprocess.get_dissimilarity_map(TanimotoIndex, title = '', filename = 'heatmap_Tanimoto') # show the heat map

In [7]:
# show importance map
initial_data = preprocess.data_load(filename = "5.0-hkg-MD-ActCoeff-processed-deletedNonZeros.csv")
#preprocess.get_importance_feature_map(initial_data, approach = 'pearson') #approach can be {'pearson', 'kendall', 'spearman'}


In [8]:
## featurization of data
data = preprocess.data_load(filename = "5.0-hkg-MD-ActCoeff-processed.csv")
## assign X, y
(X, y, MFprints) = preprocess.featurizer(data, polymers_dict,  'solvation') # descr = 'solvation' or 'activity'
## split and normalize data
(data_solvation_scaler, X_train, X_test, y_train, y_test) = preprocess.normalizedata(X, y, splitRatio=0.2, state = 48, transform = True, property = 'Y')
print(f"X_train, X_test, y_train, & y_test has shapes of {X_train.shape}, {X_test.shape}, {y_train.shape}, & {y_test.shape} respectively.")

X_train, X_test, y_train, & y_test has shapes of (73, 137), (19, 137), (73, 12), & (19, 12) respectively.


In [9]:
# transform features using PCA
print("***********************A1*************************")
pca_X_train_MDFP, pca_X_test_MDFP = preprocess.convertInputsPCA(X_train, X_test, MFprints, type = 'MDFP', useTrained = True)
print(f"pca_X_train_MDFP & pca_X_test_MDFP has shapes of {pca_X_train_MDFP.shape}, and {pca_X_test_MDFP.shape}, respectively.")
#
print("***********************A2*************************")
pca_X_train_ALL, pca_X_test_ALL = preprocess.convertInputsPCA(X_train, X_test, MFprints, type = 'All', useTrained = True)
print(f"pca_X_train_ALL & pca_X_test_ALL has shapes of {pca_X_train_ALL.shape}, and {pca_X_test_ALL.shape}, respectively.")


***********************A1*************************


UnboundLocalError: local variable 'length_MDFP' referenced before assignment

# Use model

In [ ]:
from src.models import trainedMODELs

In [ ]:
model = trainedMODELs()

In [ ]:
### SVR
# A0
SVR_A0_results_train, SVR_A0_results_test = model.implementation(model = 'SVR', type_data = 'A0', xtrain = X_train, xtest = X_test)
# A1
SVR_A1_results_train, SVR_A1_results_test = model.implementation(model = 'SVR', type_data = 'A1', xtrain = pca_X_train_MDFP, xtest = pca_X_test_MDFP)
# A2
SVR_A2_results_train, SVR_A2_results_test = model.implementation(model = 'SVR', type_data = 'A2', xtrain = pca_X_train_ALL, xtest = pca_X_test_ALL)

### ANN
# A0
ANN_A0_results_train, ANN_A0_results_test = model.implementation(model = 'ANN', type_data = 'A0', xtrain = X_train, xtest = X_test)
# A1
ANN_A1_results_train, ANN_A1_results_test = model.implementation(model = 'ANN', type_data = 'A1', xtrain = pca_X_train_MDFP, xtest = pca_X_test_MDFP)
# A2
ANN_A2_results_train, ANN_A2_results_test = model.implementation(model = 'ANN', type_data = 'A2', xtrain = pca_X_train_ALL, xtest = pca_X_test_ALL)

### RFR
# A0
RFR_A0_results_train, RFR_A0_results_test = model.implementation(model = 'RFR', type_data = 'A0', xtrain = X_train, xtest = X_test)
# A1
RFR_A1_results_train, RFR_A1_results_test = model.implementation(model = 'RFR', type_data = 'A1', xtrain = pca_X_train_MDFP, xtest = pca_X_test_MDFP)
# A2
RFR_A2_results_train, RFR_A2_results_test = model.implementation(model = 'RFR', type_data = 'A2', xtrain = pca_X_train_ALL, xtest = pca_X_test_ALL)

In [ ]:
# plot box plot
model.plot_box_MAE()

In [ ]:
def box_plot_MAE_ALL(ytrain, ytest, ytrain_pred_list, ytest_pred_list, type_data, title):
    
    # load library
    import string
    
    # plot figures
    fig, axs = plt.subplots(3, 2, figsize=(10, 10))

    axs[0, 0].grid(False); axs[0, 1].grid(False)
    axs[1, 0].grid(False); axs[1, 1].grid(False)
    axs[2, 0].grid(False); axs[2, 1].grid(False)

    ytrain = ytrain.iloc[:, :ytrain.shape[1]]; ytest = ytest.iloc[:, :ytest.shape[1]]
    
    box_00 = axs[0, 0].boxplot((np.abs(ytrain.to_numpy() - ytrain_pred_list[0].to_numpy())), patch_artist = True, notch = False, showfliers = False)
    box_01 = axs[0, 1].boxplot((np.abs(ytest.to_numpy() - ytest_pred_list[0].to_numpy())), patch_artist = True, notch = False, showfliers = False)

    box_10 = axs[1, 0].boxplot((np.abs(ytrain.to_numpy() - ytrain_pred_list[1].to_numpy())), patch_artist = True, notch = False, showfliers = False)
    box_11 = axs[1, 1].boxplot((np.abs(ytest.to_numpy() - ytest_pred_list[1].to_numpy())), patch_artist = True, notch = False, showfliers = False)

    box_20 = axs[2, 0].boxplot((np.abs(ytrain.to_numpy() - ytrain_pred_list[2].to_numpy())), patch_artist = True, notch = False, showfliers = False)
    box_21 = axs[2, 1].boxplot((np.abs(ytest.to_numpy() - ytest_pred_list[2].to_numpy())), patch_artist = True, notch = False, showfliers = False)

    # set colors
    colors = [
                '#0000FF', '#00FF00', '#FFFF00', '#DE3163', 
                '#00FFFF', '#00FFF0', '#6495ED', '#FFF0FF', 
                '#000FFF', '#0FFF00', '#FFFF0F', '#FF7F50', 
                #'#F000FF', '#FFBF00'
            ]
    for patch, color in zip(box_00['boxes'], colors): patch.set_facecolor(color)
    for patch, color in zip(box_01['boxes'], colors): patch.set_facecolor(color)
    for patch, color in zip(box_10['boxes'], colors): patch.set_facecolor(color)
    for patch, color in zip(box_11['boxes'], colors): patch.set_facecolor(color)
    for patch, color in zip(box_20['boxes'], colors): patch.set_facecolor(color)
    for patch, color in zip(box_21['boxes'], colors): patch.set_facecolor(color)

    # update the text formats
    plt.rcParams['mathtext.fontset'] = 'cm'
    plt.rcParams["font.family"] = "serif"
    plt.rcParams["font.serif"] = ["Times New Roman"] + plt.rcParams["font.serif"]
    plt.rcParams['font.size'] = 10

    # set labels
    fig.supylabel(r'$\rm MAE $', fontsize = 22, fontweight = 'bold')
    fig.supxlabel(r'$\rm Solvation \ descriptors $', fontsize = 22, fontweight = 'bold')

    axs[0, 0].set_title(r'$\rm Train$', fontsize = 22, fontweight = 'bold')
    axs[0, 1].set_title(r'$\rm Test$', fontsize = 22, fontweight = 'bold')

    alphabets = list(string.ascii_uppercase)[:len(list(ytrain.columns))]
    axs[2, 0].set_xticklabels(labels=[f'{i}' for i in alphabets], fontsize='x-large') 
    axs[2, 1].set_xticklabels(labels=[f'{i}' for i in alphabets], fontsize='x-large') 

    axs[0, 0].set_ylabel(r'$SVR$', labelpad = 5, fontsize = 15, fontweight='bold')
    axs[1, 0].set_ylabel(r'$ANN$', labelpad = 5, fontsize = 15, fontweight='bold')
    axs[2, 0].set_ylabel(r'$RFR$', labelpad = 5, fontsize = 15, fontweight='bold')

    # set axis
    axs[0, 0].set_xticks([]); axs[1, 0].set_xticks([])
    axs[0, 1].set_xticks([]); axs[1, 1].set_xticks([])

    # if type_data == 'A0':
    #     axs[0, 0].set_ylim([-0.02, 0.3]); axs[1, 0].set_ylim([-0.02, 0.3]); axs[2, 0].set_ylim([-0.02, 0.3])
    #     axs[0, 1].set_ylim([-0.02, 0.7]); axs[1, 1].set_ylim([-0.02, 0.7]); axs[2, 1].set_ylim([-0.02, 0.7])

    # if type_data == 'A1':
    #     axs[0, 0].set_ylim([-0.02, 0.5]); axs[1, 0].set_ylim([-0.02, 0.5]); axs[2, 0].set_ylim([-0.02, 0.5])
    #     axs[0, 1].set_ylim([-0.02, 1.0]); axs[1, 1].set_ylim([-0.02, 1.0]); axs[2, 1].set_ylim([-0.02, 1.0])
    
    # if type_data == 'A2':
    #     axs[0, 0].set_ylim([-0.02, 0.6]); axs[1, 0].set_ylim([-0.02, 0.6]); axs[2, 0].set_ylim([-0.02, 0.6])
    #     axs[0, 1].set_ylim([-0.02, 1.5]); axs[1, 1].set_ylim([-0.02, 1.5]); axs[2, 1].set_ylim([-0.02, 1.5])
   

    plt.tight_layout()
    plt.show
    #plt.savefig(f'reports/figures/mainPAPER/{title}.png', dpi=600) 

In [ ]:
#plot figures
#model.
box_plot_MAE_ALL(ytrain = y_train, 
                        ytest = y_test, 
                            ytrain_pred_list = [SVR_A0_results_train, ANN_A0_results_train, RFR_A0_results_train],
                                ytest_pred_list = [SVR_A0_results_test, ANN_A0_results_test, RFR_A0_results_test],
                                type_data = 'A0', 
                                title = 'performance_A0_MAE_all_models_no_diffusion'
                            )